## Model Building 

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
item_categories = pd.read_csv("data/item_categories.csv")
items = pd.read_csv("data/items.csv")
shops = pd.read_csv("final_shops.csv")
sales = pd.read_csv("sales.csv")
test = pd.read_csv("data/test.csv")

We'd try different possible baselines

Predictions based on October 2015

In [14]:
interim= sales[sales["date_block_num"]==33].groupby(["shop_id", "item_id"],as_index=False).sum()[["shop_id","item_id","item_cnt_day"]]
interim["item_cnt_day"].clip(0,20,inplace=True)
interim2 = pd.merge(test,interim,how = 'left',left_on = ["shop_id","item_id"],right_on = ['shop_id','item_id'])
interim2

,ID,shop_id,item_id,item_cnt_day
0,0,5,5037,NaN
1,1,5,5320,NaN
2,2,5,5233,1.0
3,3,5,5232,NaN
4,4,5,5268,NaN
...,...,...,...,...
214195,214195,45,18454,1.0
214196,214196,45,16188,NaN
214197,214197,45,15757,NaN
214198,214198,45,19648,NaN


In [15]:
interim2 = interim2[["ID","item_cnt_day"]]

In [16]:
interim2.columns = ["ID","item_cnt_month"]

In [17]:
interim2.fillna(0,inplace = True)

In [18]:
interim2.to_csv("sub2.csv",index = False)

A score of 1.167777!

Testing the hypothesis of seasonal trend in items and thereby using values from Nov 2014.

In [20]:
interim2 = sales[sales.date_block_num == 22].groupby(["shop_id","item_id"],as_index = False).sum()[["shop_id","item_id","item_cnt_day"]]

In [21]:
interim2['item_cnt_day'].clip(0,20,inplace = True)

In [22]:
interim2 = pd.merge(test,interim2,how = "left",left_on = ["shop_id","item_id"],right_on = ["shop_id","item_id"])
interim2 = interim2[["ID","item_cnt_day"]]
interim2.columns = ["ID","item_cnt_month"]
interim2.fillna(0,inplace = True)
interim2.to_csv("sub3.csv",index = False)

The score for this is 1.60 thereby indicating the presence of FMCG.

In such scenario we can make predictions based on recent trends.

In [24]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,items_english,meta_category,shop_name,shops_english,town,revenue,year
0,2013-01-02,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,Kino - Blu-Ray,Kino,"Ярославль ТЦ ""Альтаир""","Aroslavl_ TZ ""Al_tair""",Aroslavl_,999.00,2013
1,2013-01-03,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,Muzyka - Vinil,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,899.00,2013
2,2013-01-05,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,Muzyka - Vinil,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,-899.00,2013
3,2013-01-06,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,Muzyka - Vinil,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,1709.05,2013
4,2013-01-15,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,Muzyka - CD firmennogo proizvodstva,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,1099.00,2013


In [26]:
interim5=sales.groupby(["item_id","shop_id","date_block_num"],as_index=False).sum()[["item_id","shop_id","date_block_num","item_cnt_day"]].groupby(["item_id","shop_id"],as_index=False).last()

In [28]:
interim5["item_cnt_day"].clip(0,20,inplace = True)
interim6 = pd.merge(test,interim5,how = "left",left_on = ["shop_id","item_id"],right_on = ["shop_id","item_id"])
interim6.fillna(0,inplace = True)
interim6 = interim6[["ID","item_cnt_day"]]
interim6.columns = ["ID","item_cnt_month"]
interim6

,ID,item_cnt_month
0,0,1.0
1,1,0.0
2,2,1.0
3,3,1.0
4,4,0.0
...,...,...
214195,214195,1.0
214196,214196,0.0
214197,214197,1.0
214198,214198,0.0


In [29]:
interim6.to_csv("sub4.csv",index = False)

1.387!

In [4]:
sales

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,items_english,meta_category,shop_name,shops_english,town,revenue,year
0,2013-01-02,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,Kino - Blu-Ray,Kino,"Ярославль ТЦ ""Альтаир""","Aroslavl_ TZ ""Al_tair""",Aroslavl_,999.00,2013
1,2013-01-03,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,Muzyka - Vinil,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,899.00,2013
2,2013-01-05,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,Muzyka - Vinil,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,-899.00,2013
3,2013-01-06,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,Muzyka - Vinil,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,1709.05,2013
4,2013-01-15,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,Muzyka - CD firmennogo proizvodstva,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,1099.00,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0,V/A Nu Jazz Selection (digipack),55,Музыка - CD локального производства,Muzyka - CD lokal_nogo proizvodstva,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,299.00,2015
2935845,2015-10-09,33,25,7460,299.00,1.0,V/A The Golden Jazz Collection 1 2CD,55,Музыка - CD локального производства,Muzyka - CD lokal_nogo proizvodstva,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,299.00,2015
2935846,2015-10-14,33,25,7459,349.00,1.0,V/A The Best Of The 3 Tenors,55,Музыка - CD локального производства,Muzyka - CD lokal_nogo proizvodstva,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,349.00,2015
2935847,2015-10-22,33,25,7440,299.00,1.0,V/A Relax Collection Planet MP3 (mp3-CD) (jewel),57,Музыка - MP3,Muzyka - MP3,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,299.00,2015


In [3]:
sales['date'] = pd.to_datetime(sales['date'], format='%Y-%m-%d')

In [4]:
sales['month'] = sales['date'].dt.month

In [20]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,items_english,meta_category,shop_name,shops_english,town,revenue,year,month
0,2013-01-02,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,Kino - Blu-Ray,Kino,"Ярославль ТЦ ""Альтаир""","Aroslavl_ TZ ""Al_tair""",Aroslavl_,999.00,2013,1
1,2013-01-03,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,Muzyka - Vinil,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,899.00,2013,1
2,2013-01-05,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,Muzyka - Vinil,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,-899.00,2013,1
3,2013-01-06,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,Muzyka - Vinil,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,1709.05,2013,1
4,2013-01-15,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,Muzyka - CD firmennogo proizvodstva,Muzyka,"Москва ТРК ""Атриум""","Moskva TRK ""Atrium""",Moskva,1099.00,2013,1


In [5]:
df=pd.DataFrame()
df=sales.groupby(["date_block_num","shop_id","item_id","month","item_price","item_category_id","meta_category","town"],as_index=False).sum()[["date_block_num","shop_id","item_id","month","item_price","item_category_id","meta_category","town","item_cnt_day"]]

In [6]:
df=df.rename(columns = {'item_cnt_day':'item_cnt_month'})

- date_block_num: numerical 
- shop_id: categorical
- item_id: categorical
- item_price: numerical
- month: categorical
- item_category_id: categorical
- meta_category: categorical
- town: categorical
- item_cnt_month: numerical

In [27]:
df.head()

,date_block_num,shop_id,item_id,month,item_price,item_category_id,meta_category,town,item_cnt_month
0,0,0,32,1,221.0,40,Kino,!Akutsk,6.0
1,0,0,33,1,347.0,37,Kino,!Akutsk,3.0
2,0,0,35,1,247.0,40,Kino,!Akutsk,1.0
3,0,0,43,1,221.0,40,Kino,!Akutsk,1.0
4,0,0,51,1,127.0,57,Muzyka,!Akutsk,1.0


In [7]:
from sklearn.preprocessing import LabelEncoder

We'd try different encoding schemes covered in the course and evaluate their performance on Linear Regression.

In [8]:
le = LabelEncoder()
le.fit(df.meta_category)
df['meta_category'] = le.transform(df.meta_category)

le.fit(df.town)
df['town'] = le.transform(df.town)

df.head()

,date_block_num,shop_id,item_id,month,item_price,item_category_id,meta_category,town,item_cnt_month
0,0,0,32,1,221.0,40,13,0,6.0
1,0,0,33,1,347.0,37,13,0,3.0
2,0,0,35,1,247.0,40,13,0,1.0
3,0,0,43,1,221.0,40,13,0,1.0
4,0,0,51,1,127.0,57,15,0,1.0


Fitting Linear Regression!

In [9]:
Xtrain=df.drop("item_cnt_month", axis=1)
# Reason for dropping item_price explained below
ytrain=df["item_cnt_month"]

Xtrain.fillna(0, inplace=True)

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(Xtrain,ytrain)
ypred = lr.predict(Xtrain)
print(np.sqrt(mean_squared_error(ytrain,ypred)))

7.611730670923669


7.6117! Worse than the baseline models or un-models, and that too on the train set.

In [15]:
lr.coef_

array([-6.02751709e-03,  4.29243409e-04,  1.17313171e-05,  4.13813305e-02,
        2.38842392e-05,  4.88228370e-02, -2.40672822e-01, -5.87388955e-03])

In [16]:
Xtrain.columns

Index(['date_block_num', 'shop_id', 'item_id', 'month', 'item_price',
       'item_category_id', 'meta_category', 'town'],
      dtype='object')

We have label encoded many features and we can't really expect linear models to give good results. Let's try some trees.

In [19]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()
dtr.fit(Xtrain,ytrain)
ypred = dtr.predict(Xtrain)
print(np.sqrt(mean_squared_error(ytrain,ypred)))

0.0


Aha! Overfitting at its best!

In order to use the Test set, we'd have to blow it up. Add columns such as town, item_category, price, town, item_cnt_month etc.

In [10]:
test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [14]:
test1 = pd.merge(test,Xtrain[["shop_id","town"]].drop_duplicates(),how = "left",on = "shop_id")
test1.shape

(214200, 4)

In [15]:
test2 = pd.merge(test1,Xtrain[["item_id","item_category_id","meta_category"]].drop_duplicates(),how = "left",on="item_id")
test2.shape

(214200, 6)

In [19]:
part1 = pd.merge(test2,Xtrain[["item_id","shop_id","item_price"]][Xtrain["date_block_num"]==33].drop_duplicates().groupby(["item_id","shop_id"],as_index=False).max(), how="left",on=["shop_id","item_id"]).dropna()
part2 = pd.merge(test2,Xtrain[["item_id","shop_id","item_price"]][Xtrain["date_block_num"]==32].drop_duplicates(), how="left",on=["shop_id","item_id"]).dropna()
part3 = pd.merge(test2,Xtrain[["item_id","shop_id","item_price"]][Xtrain["date_block_num"]==31].drop_duplicates(), how="left",on=["shop_id","item_id"]).dropna()
part4 = pd.merge(test2,Xtrain[["item_id","shop_id","item_price"]][Xtrain["date_block_num"]==30].drop_duplicates(), how="left",on=["shop_id","item_id"]).dropna()
part5 = pd.merge(test2,Xtrain[["item_id","shop_id","item_price"]][Xtrain["date_block_num"]==29].drop_duplicates(), how="left",on=["shop_id","item_id"]).dropna()
part6 = pd.merge(test2,Xtrain[["item_id","shop_id","item_price"]][Xtrain["date_block_num"]==28].drop_duplicates(), how="left",on=["shop_id","item_id"]).dropna()
# half a year should be enough

# For non-existing pairs lets use the average price of items
part7 = pd.merge(test2,Xtrain[["item_id","item_price"]].drop_duplicates().groupby("item_id").mean(), how="left",on=["item_id"]).dropna()

# Now join together. Tricky. Only one value needed. Strict order. Part 1 better than part2, i.e. if 
# part 1 exists dont add part 2 and so forth
prices=part1
print("After adding month -1:")
print(prices.shape)
ids=prices.item_id
prices=pd.concat([prices,part2[~part2.item_id.isin(ids)]])
print("After adding month -2:")
print(prices.shape)
ids=prices.item_id
prices=pd.concat([prices,part3[~part3.item_id.isin(ids)]])
print("After adding month -3:")
print(prices.shape)
ids=prices.item_id
prices=pd.concat([prices,part4[~part4.item_id.isin(ids)]])
print("After adding month -4:")
print(prices.shape)
ids=prices.item_id
prices=pd.concat([prices,part5[~part5.item_id.isin(ids)]])
print("After adding month -5:")
print(prices.shape)
ids=prices.item_id
prices=pd.concat([prices,part6[~part6.item_id.isin(ids)]])
ids=prices.item_id
print("After adding month -6:")
print(prices.shape)

prices["shop_item_pair_id"]=prices.shop_id.astype(str)+"-"+prices.item_id.astype(str)
part7["shop_item_pair_id"]=part7.shop_id.astype(str)+"-"+part7.item_id.astype(str)
prices=pd.concat([prices,part7[~part7.shop_item_pair_id.isin(prices.shop_item_pair_id)]])
prices.drop("shop_item_pair_id", axis=1,inplace=True)
print("After adding prices only according to item_id:")
print(prices.shape)

# Merge with original DF to include also the 15k lines with unknown items
final_test=pd.merge(test2, prices[["shop_id","item_id","item_price"]], how="left",on=["shop_id","item_id"])
final_test=final_test.groupby(["ID"],as_index=False).max()

# Lets add the month and price_category
final_test["date_block_num"]=34
final_test["month"]=11

After adding month -1:
(28680, 7)
After adding month -2:
(29197, 7)
After adding month -3:
(29354, 7)
After adding month -4:
(29449, 7)
After adding month -5:
(29513, 7)
After adding month -6:
(29558, 7)
After adding prices only according to item_id:
(198963, 7)


In [20]:
final_test.shape

(214200, 9)

In [21]:
print(final_test[final_test["item_price"].isnull()].head())
# 15k lines of new items. Fill with global price median
final_test.item_price.fillna(final_test.item_price.median(),inplace=True)
final_test.item_category_id.fillna(final_test.item_category_id.median(),inplace=True)
final_test.meta_category.fillna(final_test.meta_category.median(),inplace=True)
print(final_test[final_test["item_price"].isnull()])
print(final_test.shape)

    ID  shop_id  item_id  town  item_category_id  meta_category  item_price  \
1    1        5     5320    28               NaN            NaN         NaN   
4    4        5     5268    28               NaN            NaN         NaN   
45  45        5     5826    28               NaN            NaN         NaN   
64  64        5     3538    28               NaN            NaN         NaN   
65  65        5     3571    28               NaN            NaN         NaN   

    date_block_num  month  
1               34     11  
4               34     11  
45              34     11  
64              34     11  
65              34     11  
Empty DataFrame
Columns: [ID, shop_id, item_id, town, item_category_id, meta_category, item_price, date_block_num, month]
Index: []
(214200, 9)


In [22]:
Xtrain.columns,final_test.columns

(Index(['date_block_num', 'shop_id', 'item_id', 'month', 'item_price',
        'item_category_id', 'meta_category', 'town'],
       dtype='object'),
 Index(['ID', 'shop_id', 'item_id', 'town', 'item_category_id', 'meta_category',
        'item_price', 'date_block_num', 'month'],
       dtype='object'))

In [23]:
final_test = final_test[["ID","date_block_num","shop_id","item_id",'month',"item_price","item_category_id","meta_category","town"]]

In [24]:
final_test.columns

Index(['ID', 'date_block_num', 'shop_id', 'item_id', 'month', 'item_price',
       'item_category_id', 'meta_category', 'town'],
      dtype='object')

In [32]:
ytrain

0          6.0
1          3.0
2          1.0
3          1.0
4          1.0
          ... 
1739017    6.0
1739018    2.0
1739019    1.0
1739020    1.0
1739021    1.0
Name: item_cnt_month, Length: 1739022, dtype: float64

In [25]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=10)
rfr.fit(Xtrain,ytrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [63]:
from sklearn.metrics import mean_squared_error

i = 0
rfr = RandomForestRegressor(n_estimators=20)
Xt = Xtrain[Xtrain['date_block_num']<((i*12)+10)]
yt = ytrain[Xt.index]
Xval = Xtrain[Xtrain['date_block_num'] == ((i*12)+10)]
yval = ytrain[Xval.index]
rfr.fit(Xt,yt)
ypred = rfr.predict(Xt)
print(np.sqrt(mean_squared_error(yt,ypred)))

1.436471772496291


In [61]:
import xgboost as xgb
param = {'max_depth':10, 
         'subsample':0.5,
         'min_child_weight':1,
         'eta':0.5, 
         'num_round':1000, 
         'seed':1,
         'silent':0,
         'eval_metric':'rmse'}

progress = dict()
i = 0
Xt = Xtrain[Xtrain['date_block_num']<((i*12)+10)]
yt = ytrain[Xt.index]
Xval = Xtrain[Xtrain['date_block_num'] == ((i*12)+10)]
yval = ytrain[Xval.index]
xgbtrain = xgb.DMatrix(Xt, yt)
watchlist  = [(xgbtrain,'train-rmse')]

bst = xgb.train(param, xgbtrain)
preds = bst.predict(xgb.DMatrix(Xval))

print(np.sqrt(mean_squared_error(preds,yval)))

[11:42:08] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1588600955503/work/src/learner.cc:328: 
Parameters: { num_round, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


4.987439128915999


In [62]:
i = 1
Xt = Xtrain[Xtrain['date_block_num']<((i*12)+10)]
yt = ytrain[Xt.index]
Xval = Xtrain[Xtrain['date_block_num'] == ((i*12)+10)]
yval = ytrain[Xval.index]
xgbtrain = xgb.DMatrix(Xt, yt)
watchlist  = [(xgbtrain,'train-rmse')]

bst = xgb.train(param, xgbtrain)
preds = bst.predict(xgb.DMatrix(Xval))

print(np.sqrt(mean_squared_error(yval,preds)))

[11:42:13] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1588600955503/work/src/learner.cc:328: 
Parameters: { num_round, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


8.16491140182803


In [26]:
ypred = rfr.predict(final_test.drop("ID",axis = 1))

In [28]:
ypred.shape

(214200,)

In [29]:
sub = pd.DataFrame()
sub["ID"] = [i for i in range(214200)]
sub["item_cnt_month"] = ypred

In [30]:
sub.to_csv("sub5.csv",index = False)

,ID,item_cnt_month
0,0,1.0
1,1,1.0
2,2,1.0
3,3,1.2
4,4,1.0
...,...,...
214195,214195,1.0
214196,214196,1.5
214197,214197,1.2
214198,214198,1.0
